In [1]:
from aimet_torch.quantsim import QuantizationSimModel, QuantScheme
import torch
from aimet_torch.model_preparer import prepare_model

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dummy_input = torch.rand(1, 3, 640, 640).to(DEVICE)
# BASE_PATH = "/teamspace/studios/this_studio/mmdetection/rtm_weights/gpu19aug/rtm_det_gpu_16Aug_rangelearning_20240819_155611_300"
# BASE_PATH = "/teamspace/studios/this_studio/mmdetection/rtm_weights/gpu19aug/rtm_det_gpu_16Aug_rangelearning_20240819_155611_300"
# BASE_PATH = "/teamspace/studios/this_studio/mmdetection/rtm_weights/gpu20aug_neck/rtm_det_gpu_20aug_20240820_073659_300"
# BASE_PATH = "/teamspace/studios/mm-det/mmdetection/rtm_weights/gpu20aug_neck_range/rtm_det_gpu_20aug_20240820_100350_350"
BASE_PATH = "/teamspace/studios/this_studio/aimet/Examples/torch/quantization/quant_scheme_W@tf / A@tf"
# BASE_PATH = "/teamspace/studios/this_studio/aimet/full_model_sim"
graph_model = torch.load(f"{BASE_PATH}/rtm_det.pth", map_location=DEVICE)

# quant_scheme = QuantScheme.training_range_learning_with_tf_enhanced_init
# quant_scheme = QuantScheme.post_training_tf_enhanced
quant_scheme = QuantScheme.post_training_tf
sim = QuantizationSimModel(model=graph_model,
                        quant_scheme=quant_scheme,
                        dummy_input=dummy_input,
                        default_output_bw=8,
                        default_param_bw=8,)
sim.load_encodings(f"{BASE_PATH}/rtm_det_torch.encodings")

2024-08-21 13:07:36,766 - root - INFO - AIMET
2024-08-21 13:07:57,650 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.10/dist-packages/aimet_common/quantsim_config/default_config.json
2024-08-21 13:07:57,686 - Quant - INFO - Unsupported op type Squeeze
2024-08-21 13:07:57,688 - Quant - INFO - Unsupported op type Mean
2024-08-21 13:07:57,705 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


### Made changes in load_encodings function to change all Quantizers to active


In [2]:
from aimet_torch.qc_quantize_op import QcQuantizeOpMode
name, module = list(sim.model.named_children())[0]

# list(list(list(module.named_children())[0][1].named_children())[0][1].named_children())[0][1].set_mode(QcQuantizeOpMode.ACTIVE)
list(list(list(module.named_children())[0][1].named_children())[0][1].named_children())[0][1]._mode

<QcQuantizeOpMode.ACTIVE: 3>

In [3]:
import shutil
import os
shutil.rmtree("/teamspace/studios/this_studio/aimet/temp", ignore_errors=True)
os.makedirs("/teamspace/studios/this_studio/aimet/temp", exist_ok=True)

sim.export(path="/teamspace/studios/this_studio/aimet/temp",
    filename_prefix="rtm_det",
    dummy_input=dummy_input.cpu(),
    use_embedded_encodings=True)

2024-08-21 13:07:57,868 - Quant - WARNING - Exporting encodings to yaml will be deprecated in a future release. Ensure that your code can work with the exported files ending in ".encodings" which are saved using json format. For the time being, if yaml export is needed, set aimet_common.utils.SAVE_TO_YAML to True.
2024-08-21 13:07:58,948 - Quant - INFO - Quantizing tensor torch.Size([1, 3, 640, 640])


NameError: name 'self' is not defined